<a href="https://colab.research.google.com/github/yukinaga/langchain/blob/main/section_3/04_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memory
Memoryは過去のやり取りを記憶し、それに基づいた回答を生成するために使用します。    
今回のコードは、以下の公式ドキュメントを参考にしています。  
https://python.langchain.com/docs/


## ライブラリのインストール
lanchain、langchain-coreとlangchain-openaiをインストールします。  

In [ ]:
!pip install langchain langchain-core langchain-openai

## API Keyの設定
OpneAIのAPIを使用するために必要な「API key」を設定します。  
以下のコードの、  
`os.environ["OPENAI_API_KEY"] = "Your API key"`  
における  
`Your API key`の箇所を、自分のAPI keyに置き換えます。  
OpenAIのAPI keyは、OpenAIのサイトで取得できます。   
https://platform.openai.com/account/api-keys


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "Your API key"

API keyの流出にはリスクがあります。  
他者に知られないように、慎重に扱ってください。

## Agentの設定
Agentを設定します。  

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import load_tools
from langchain.agents import create_openai_functions_agent

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは優秀なアシスタントです。"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

tools = load_tools(
    tool_names=["llm-math"],
    llm=chat
)

agent = create_openai_functions_agent(chat, tools, prompt)

## Memory利用
以下のコードは、Memoryの利用例の一つです。  
「AgentExecutor」の使用時に設定します。  

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

これまでのやりとりを、`hisotory`に設定します。

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

history = [
    HumanMessage(content="僕の名前はAI太郎。"),
    AIMessage(content="こんにちは、AI太郎。何かお助けできますか？。"),
    ]

以下の質問をすると、これまでのやりとりをもとに返答が作成されます。

In [ ]:
agent_executor.invoke({
    "chat_history": history,
    "input": "僕の名前は何ですか？"
    })

コードの実行後は、OpenAIのサイトでAPIの使用量を確認しましょう。  
https://platform.openai.com/account/usage